# NIRISS AMI: Pipeline

**Use case:** Run pipeline on NIRISS AMI simulated.<br>
**Data:** JWST simulated NIRISS data from MIRAGE.<br>
**Tools:**  mirage, jwst, astropy.<br>
**Cross-intrument:** <br>
**Documentation:** This notebook is part of a STScI's larger [post-pipeline Data Analysis Tools Ecosystem](https://jwst-docs.stsci.edu/jwst-post-pipeline-data-analysis).<br>

## Introduction
This notebook runs JWST pipeline on Aperture Masking Interferometry(AMI) data of binary point source AB Dor and calibrator HD37093 simulated with Mirage.

Steps:

[1] Run Detector1 pipeline on all _uncal.fits files to create _rate.fits and _rateints.fits files.

[2] Run Image2 pipeline on all _rate.fits files to create _cal.fits and on _rateints.fits files to
   create _calints.fits files.

 
[3] Run ImPlaneIA ([Greenbaum, A. et al. 2015](https://ui.adsabs.harvard.edu/abs/2015ApJ...798...68G/abstract)) to extract observables in oifits format.    

*Developer Note:*
Create the conda environment needed to run this notebook by issuing the following command:
```
conda create -n <myenv> python=3.6
```
Activate the environment and then open the notebook.

```
conda activate <myenv>
jupyter notebook
```

***

## Imports
Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *matplotlib.pyplot* for plotting data
- *zipfile* for accessing zip file
- *urllib.request* to access URL
- *jwst.pipeline Detector1Pipeline, Image2Pipeline* for calibrating raw data
- ImplaneIA to extract interferometric obssrvables from calibrated data

In [ ]:
%matplotlib inline
import os
import sys
import glob
import time

import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy import units as u
import zipfile
import urllib.request

from jwst.pipeline import Detector1Pipeline, Image2Pipeline

from nrm_analysis.misctools import utils
import nrm_analysis
from nrm_analysis.fringefitting.LG_Model import NRM_Model
from nrm_analysis import nrm_core, InstrumentData
from nrm_analysis import find_affine2d_parameters as FAP
from nrm_analysis.misctools.implane2oifits import calibrate_oifits

from nrm_analysis.misctools.utils import Affine2d

*Developer Note:*
Plese follow the instructions on https://webbpsf.readthedocs.io/en/latest/installation.html to download WebbPSF data 
files and create WEBBPSF_PATH location.

<div class="alert alert-info">

**Note:** DO NOT UPGRADE PYSIAF AS INSTRUCTED BY THE ABOVE WARNING. 
Pysiaf 0.10.0 uses PRD release PRDOPSSOC-031. APT 2020.4.1 that was used to create the xml and pointing files used in this notebook uses PRDOPSSOC-030. The mismatch bewtween these two PRD versions causes incorrect placement of PSF. The version of Mirage used in this notebook comes with pysiaf 0.10.0. It was downgraded to 0.9.0 using the file requirements.txt to resolve the placement issue. Using Pysiaf 0.10.0 will cause incorrect placement of the PSF.

</div>

In [ ]:
nrm_analysis.__path__

In [ ]:
%pwd

## Loading data
Download simulated data created by 1_niriss_ami_binary.ipynb and reference files needed to calibrate data without bad pixels.

In [ ]:
boxlink = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/niriss_ami_binary/niriss_ami_binary2.zip'
boxfile = './niriss_ami_binary2.zip'

# Download zip file
if not os.path.exists(boxfile):
    urllib.request.urlretrieve(boxlink, boxfile)

    zf = zipfile.ZipFile(boxfile, 'r')
    zf.extractall()

In [ ]:
# Define directory that has existing Mirage simulations
currentdir = os.getcwd()
mirage_sim_dir = os.path.join(currentdir, 'mirage_sim_data/')
print(mirage_sim_dir)
datafiles = sorted(glob.glob(mirage_sim_dir + 'jw*uncal.fits'))
print(datafiles)

## Examine the input raw files

In [ ]:
data = []
for i, df in enumerate(datafiles):
    file = fits.open(df)
    file.info()
    im = file[1].data
    print(im[0].shape)
    data.append(im[0])
print(data[0].shape, data[1].shape)
f = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("AB-Dor")
plt.imshow(data[0][4], origin='lower')
plt.subplot(1, 2, 2)
plt.title("HD37093")
plt.imshow(data[1][11], origin='lower')

## Define output directory and non-default reference files directory

Note that we will not use the cfgfiles but update the steps and override reference files using the .run() method.

In [ ]:
# Define output directory to save pipeline output products and default configuration files.
odir = './pipeline_calibrated_data/'
if not os.path.exists(odir):
    os.mkdir(odir)

# Define path to non-default reference files.
# We are using these files to calibrate data that is simulated without bad pixels.
refdir = './ref_files_non_default/'

# os.environ['CRDS_PATH']='$HOME/crds_cache'
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['CRDS_CONTEXT'] = 'jwst_0641.pmap'

## Run Detector1 and Image2 pipelines

In [ ]:
datafiles = sorted(glob.glob(mirage_sim_dir + 'jw*uncal.fits'))
print(datafiles)
for df in datafiles:
# Run Detector1, Image 2 pipelines
    result1 = Detector1Pipeline()
    superbiasfile = refdir + 'jwst_niriss_superbias_sim.fits'
    darkfile = refdir + 'jwst_niriss_dark_sub80_sim.fits'
    result1.superbias.override_superbias = superbiasfile
    result1.dark_current.override_dark = darkfile
    result1.ipc.skip = True
    result1.save_results = True
    result1.output_dir = odir
    result1.run(df)

    df_rate = odir + os.path.basename(df.replace('uncal', 'rate'))
    flatfieldfile = refdir + "jwst_niriss_flat_general.fits"
    result2 = Image2Pipeline()
    result2.flat_field.override_flat = flatfieldfile
    result2.photom.skip = True
    result2.resample.skip = True
    result2.save_results = True
    result2.output_dir = odir
    result2.run(df_rate)

    df_rateints = odir + os.path.basename(df.replace('uncal', 'rateints'))
    result3 = Image2Pipeline()
    result3.flat_field.override_flat = flatfieldfile
    result3.photom.skip = True
    result3.resample.skip = True
    result3.save_results = True
    result3.output_dir = odir
    result3.run(df_rateints)

## Examine the output files

In [ ]:
ratefiles = sorted(glob.glob(odir + 'jw*rate.fits'))
print(ratefiles)
rateintsfiles = sorted(glob.glob(odir + 'jw*rateints.fits'))
print(rateintsfiles)
calfiles = sorted(glob.glob(odir + 'jw*cal.fits'))
print(calfiles)
calintsfiles = sorted(glob.glob(odir + 'jw*calints.fits'))
print(calintsfiles)

### rate and rateints files

In [ ]:
for i, rateintf in enumerate(rateintsfiles):
    file = fits.open(rateintf)
    file.info()
for i, ratef in enumerate(ratefiles):
    file = fits.open(ratef)
    file.info()

### cal and calints files 

In [ ]:
for i, calintf in enumerate(calintsfiles):
    file = fits.open(calintf)
    file.info()
for i, calf in enumerate(calfiles):
    file = fits.open(calf)
    file.info()

## Display calibrated data

In [ ]:
data = []
for df in calfiles:
    file = fits.open(df)
    im = file[1].data
    print(im.shape)
    data.append(im)
# print(data[0].shape, data[1].shape)
f = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("AB-Dor 2D calibrated file")
plt.imshow(data[0], origin='lower')
plt.subplot(1, 2, 2)
plt.title("HD37093 2D calibrated file")
plt.imshow(data[1], origin='lower')

## Run ImPlaneIA to reduce calibrated images to fringe observables

### Define functions

In [ ]:
np.set_printoptions(precision=4, linewidth=160)

def examine_observables(ff, trim=36):
    """ input: FringeFitter instance after fringes are fit """

    print("\nExamine_observables, standard deviations & variances of *independent* CP's and CAs:")
    print("   Closure phase mean {:+.4f}  std dev {:.2e}  var {:.2e}".format(ff.nrm.redundant_cps.mean(),\
              np.sqrt(utils.cp_var(ff.nrm.N, ff.nrm.redundant_cps)), utils.cp_var(ff.nrm.N, ff.nrm.redundant_cps)))

    print("   Closure amp   mean {:+.4f}  std dev {:.2e}  var {:.2e}".format(ff.nrm.redundant_cas.mean(),\
              np.sqrt(utils.cp_var(ff.nrm.N, ff.nrm.redundant_cas)), utils.cp_var(ff.nrm.N, ff.nrm.redundant_cas)))

    print("    Fringe amp   mean {:+.4f}  std dev {:.2e}  var {:.2e}".format(ff.nrm.fringeamp.mean(),
                                                                             ff.nrm.fringeamp.std(),
                                                                             ff.nrm.fringeamp.var()))

    np.set_printoptions(precision=3, formatter={'float': lambda x: '{:+.1e}'.format(x)}, linewidth=80)
    print(" Normalized residuals central 6 pixels")
    tlo, thi = (ff.nrm.residual.shape[0]//2 - 3, ff.nrm.residual.shape[0]//2 + 3)
    print((ff.nrm.residual/ff.datapeak)[tlo:thi, tlo:thi])
    print(" Normalized residuals max and min: {:.2e}, {:.2e}".format(ff.nrm.residual.max() / ff.datapeak,
                                                                     ff.nrm.residual.min() / ff.datapeak))
    utils.default_printoptions()


def raw_observables(fitsfn=None, fitsimdir=None, affine2d=None,
                    psf_offset_find_rotation=(0.0, 0.0),
                    psf_offset_ff=None,
                    rotsearch_d=None,
                    set_pistons=None,
                    oversample=3,
                    mnem='',
                    firstfew=None,
                    verbose=False):
    """
        Reduce calibrated data to fringe observables

        returns: affine2d (measured or input),
        psf_offset_find_rotation (input),
        psf_offset_ff (input or found),
        fringe pistons/r (found)
    """

    if verbose:
        print("raw_observables: input", fitsimdir + fitsfn)
    if verbose:
        print("raw_observables: oversample", oversample)

    fobj = fits.open(fitsimdir + fitsfn)

    if verbose:
        print(fobj[0].header['FILTER'])
    niriss = InstrumentData.NIRISS(fobj[0].header['FILTER'],
                                   bpexist=False,
                                   firstfew=firstfew,# read_data truncation to only read first few slices...
                                   )


    ff = nrm_core.FringeFitter(niriss,
                               datadir=fitsimdir,
                               savedir=fitsimdir,
                               oversample=oversample,
                               oifprefix="ov{:d}_".format(oversample)+mnem,
                               interactive=False,
                               save_txt_only=False)

    ff.fit_fringes(fitsimdir + fitsfn)
    examine_observables(ff)

    np.set_printoptions(formatter={'float': lambda x: '{:+.2e}'.format(x)}, linewidth=80)
    if verbose:
        print("raw_observables: fringepistons/rad", ff.nrm.fringepistons)
    utils.default_printoptions()
    return affine2d, psf_offset_find_rotation, ff.nrm.psf_offset, ff.nrm.fringepistons


def main(fitsimdir, ifn, oversample=3, mnem='', firstfew=None, verbose=False):
    """
    fitsimdir: string: dir containing data file
    ifn: str inout file name

    """

    np.set_printoptions(formatter={'float': lambda x: '{:+.2e}'.format(x)}, linewidth=80)
    if verbose:
        print("main: ", ifn)
    if verbose:
        print("main: fitsimdir", fitsimdir)

    aff, psf_offset_r, psf_offset_ff, fringepistons = raw_observables(ifn, fitsimdir,
                                                                      oversample=oversample,
                                                                      mnem=mnem,
                                                                      firstfew=firstfew,
                                                                      verbose=verbose)
    print('aff', aff, 'psf_offset_r', psf_offset_r, 'psf_offset_ff', psf_offset_ff, 'fringepistons', fringepistons)
    del aff
    del psf_offset_r
    del psf_offset_ff
    del fringepistons

### Run ImPlaneIA

In [ ]:
mirdatafiles = ['jw01093001001_01101_00001_nis_calints.fits',
                'jw01093002001_01101_00001_nis_calints.fits',
               ]

# Choose FIRSTFEW = None to analyze all integrations
FIRSTFEW = 5
OVERSAMPLE = 7
print('FIRSTFEW', FIRSTFEW, 'OVERSAMPLE', OVERSAMPLE)

datasuperdir = odir

COUNT = 0
for fnmir in mirdatafiles:
    print('\nAnalyzing\n   ', COUNT, fnmir.replace('.fits', ''), end=' ')
    hdr = fits.getheader(datasuperdir + fnmir)
    print(hdr['FILTER'], end=' ')
    print(hdr['TARGNAME'], end=' ')
    print(hdr['TARGPROP'])
    # next line for convenient use in oifits writer which looks up target online
    catname = hdr['TARGPROP'].replace('-', '') # for target lookup on-line, otherwise UNKNOWN used
    fits.setval(datasuperdir + fnmir, 'TARGNAME', value=catname)
    fits.setval(datasuperdir + fnmir, 'TARGPROP', value=catname)
    main(fitsimdir=datasuperdir,
         ifn=fnmir,
         oversample=OVERSAMPLE,
         mnem='',
         firstfew=FIRSTFEW,
         verbose=False)
    COUNT += 1

### Examine the output products

Analytical model is created and interferometric observables are calculated for each integration of the data. The output products are stored in a folder that has rootname of the file, jw01093001001_01101_00001_nis_calints for AB-dor and jw01093001001_01101_00001_nis_calints foir HD37093. 

In [ ]:
# integration 0 (1st integration)
results_int0 = glob.glob(odir + "jw01093001001_01101_00001_nis_calints/*00*")

In [ ]:
results_int0

### Information about observables calculated from the 1st integration


```
- phases_00.txt: 35 fringe phases
- amplitudes_00.txt: 21 fringe amplitudes
- CPs_00.txt: 35 closure phases
- CAs_00.txt: 35 closure amplitudes
- fringepistons_00.txt: 7 pistons (optical path delays between mask holes)
- solutions_00.txt: 44 fringe coefficients of terms in the analytical model
- modelsolution_00.fits: analytical model
- n_modelsolution_00.fits: normalized analytical model
- residual_00.fits: data - model
- n_residual_00.fits: normalized residual

```

In [ ]:
cropped_data = fits.getdata(odir + "jw01093001001_01101_00001_nis_calints/centered_0.fits")
model = fits.getdata(odir + "jw01093001001_01101_00001_nis_calints/modelsolution_00.fits")
residual = fits.getdata(odir + "jw01093001001_01101_00001_nis_calints/residual_00.fits")
n_residual = fits.getdata(odir + "jw01093001001_01101_00001_nis_calints/n_residual_00.fits")

In [ ]:
f = plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.title("AB-Dor cropped data")
plt.imshow(cropped_data, origin='lower')
plt.subplot(1, 3, 2)
plt.title("AB-Dor analytical model")
plt.imshow(model, origin='lower')
plt.subplot(1, 3, 3)
plt.title("AB-Dor residual (data - model)")
plt.imshow(residual, origin='lower')

In [ ]:
plt.title("AB-Dor normalized residual")
plt.imshow(n_residual, clim=(-0.03,0.03), origin='lower')
plt.colorbar()

### OIFITS files for the target and calibrator

OIFITS is the standard data exchange format for Optical Interferometry.  It is based on the Flexible Image Transport System (FITS).  OIFITS files include data tables for storing interferometric observables, including squared visibilities and closure phases. 

In [ ]:
odirfiles = glob.glob(odir + "*oifits")
odirfiles

### Calibrate the closure phases and fringe amplitudes of target with the closure phases and fringe amplitudes of the calibrator.

This step is necessary to remove instrumental contribution to closure phases and fringe amplitudes.

In [ ]:
# Define the target and calibrator OIFITS files
targ_oifits = glob.glob(os.path.join(odir, '*ABDOR*oifits'))[0]
cal_oifits = glob.glob(os.path.join(odir, '*HD37093*oifits'))[0]

# Produce a single calibrated OIFITS file
print("************  Running calibrate ***************")
calibrate_oifits(targ_oifits, cal_oifits, oifprefix='calib_', datadir=odir)

print("The output of calibrate is calibrated oifits file that will be used as an input to 3_niriss_ami_binary.ipynb.")

## Aditional Resources

- [JWST NIRISS AMI](https://jwst-docs.stsci.edu/near-infrared-imager-and-slitless-spectrograph/niriss-observing-modes/niriss-aperture-masking-interferometry)

## About this notebook

**Author:** Deepashri Thatte, Anand Sivaramakrishnan, Rachel Cooper,
**Updated On:** 2020-12-18 

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 